In [1]:
import pyPamtra
import datetime
import numpy as np
import pickle
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import xarray as xr
get_ipython().run_line_magic('matplotlib', 'inline')

import ERA5_Tb ##---> needed to work with PAMTRA simulated TBs
import Regression_Retrieval

In [2]:
# this creates the input for readERA5
from cdo import *

## Switches

In [3]:
run_simulation=True
take_random_spring=True
take_synth_ar_dates=True
#second_time=True
rerun_dates=True

## ERA5 Preprocessing

In [4]:
ERA5_preprocess=ERA5_Tb.ERA5_Preprocessing

## Run PAMTRA

In [5]:
PAMTRAhandler=ERA5_Tb.PAMTRA_Handler

## PAMTRA Output Plots

In [6]:
def plotDataHyd(lon,lat,data):
    data[data < 0.05] = np.nan
    
    map_proj=ccrs.Mollweide(central_longitude=-30)
    data_proj=ccrs.PlateCarree()
    ax = plt.subplot(221,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,0],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(222,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,1],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(223,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,2],transform=data_proj,cmap='jet')
    plt.colorbar()
    ax = plt.subplot(224,projection=map_proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:,3],transform=data_proj,cmap='jet')
    plt.colorbar()
    plt.show()
    return None

def plotMap(lon,lat,data):
    proj = ccrs.NorthPolarStereo(central_longitude=10)
    data_crs = ccrs.PlateCarree()
    ax = plt.axes(projection=proj)
    ax.coastlines()
    plt.pcolormesh(lon,lat,data[:,:],transform=data_crs,cmap='jet')
    plt.colorbar()

    plt.show()

    return


## Run Main Routine

In [7]:
def create_random_dates(old_dates,number_of_dates=3):
    #dates=['19820320','19900409', '19910428', '19950306', '19970322','20050418','20170425','20210331']
    #dates=['20040301', '19970410', '20060417', '19940330', '20140322', '19950410', '19990423', '20190421'] #second dates
    import random
    dates=[]
    d=0
    while d <number_of_dates:
        year=random.randint(1979,2021)
        month=random.randint(3,4)
        if month==3:
            day=random.randint(1,31)
        else:
            day=random.randint(1,30)    
        
        date=str(year*10000+month*100+day)
        if not date in old_dates:
            dates.append(date)
            d+=1
    return dates

hour_to_analyse="12" # UTC

dates=["20220315"]#["20150314","20160311","20161013","20180224","20180225","20190319","20200416","20200419"]
old_dates=["19820320",'19900409', #'19910428',
           #'19950306', '19970322',
           #'20050418',
           '20170425','20210331', "20040301",#random dates
           #, '19970410', '20060417', '19940330', '20140322', '19950410', '19990423', '20190421', #second random dates
           "20110317",
           "20110423",
           "20150314",
            "20160311","20180224",
           #"20161013",
           "20180225","20190319",
           "20200416","20200419", # synth AR dates
          ]
new_dates=[#'20190411', 
    #'20020413', 
    #'19920330', 
    #'19900312',
    #'20200420',
    #'19950404', '20210421', 
    #'20170301', '19790313','19860407', '20160411', 
    #'20140320', '19930304', '19790426',
    #'20020401', '19920415', '19820322',
    #'19840425', '19820414', '20060330'
    ]
if take_random_spring:
    if rerun_dates:
        dates=create_random_dates(old_dates,number_of_dates=15)
    else:
        pass

    
    # default 50 dates
    dates=['19790312', '19810330', '19810424', '19820320', '19820416',
           '19830316', '19830331', '19830414', '19840308', '19840413',
           '19840428', '19880330', '19890422', '19900409', '19900411', '19900418', 
           '19910301', '19910428', '19920302','19920413', '19930303', '19930430',
           '19940421', '19950306', '19950317', '19970322', '19980324', '19980409', 
           '20030415', '20050401', '20060317', '20070324',
           '20080313', '20080404', '20080411', '20080430', '20090324', '20100323', '20110311', '20110329', 
           '20110413', '20120312','20130313','20140309', '20140330', '20160409', '20180319', '20180326', 
        '20200326','20210402',#'#20220315', 
        '20220310'
           #'20220316', '20220410'
    ]
#if take_synth_ar_dates:
    #dates=["20110317","20110423","20150314",
    #       "20160311","20180224","20180225",
    #       "20190319","20200416","20200419"]
    #
#dates=['19820320', '19840308',#

#       '19900409', '19910428',
#       '19950306', '19970322', 
#       '20080313', '20110311',
#       '20110413'
#      ]
#dates=["20220315","20220316","20220410"]
#old_dates#["20220312","20220313",
      #  "20220314",
      # "20220315","20220316"
      #]#"20220312","20220313","20220314","20220315","20220316","20220320","20220321"]#new_dates
#dates=['19820416', '19880330', '20080411']
#dates=['19810424', '19840428', '19890422', '19900418', '19910301', '19930303', '19980324', '19980409', '20030415', '20060317', '20080404', '20080430', '20100323', '20140330', '20180319']
print(sorted(dates))

['19790312', '19810330', '19810424', '19820320', '19820416', '19830316', '19830331', '19830414', '19840308', '19840413', '19840428', '19880330', '19890422', '19900409', '19900411', '19900418', '19910301', '19910428', '19920302', '19920413', '19930303', '19930430', '19940421', '19950306', '19950317', '19970322', '19980324', '19980409', '20030415', '20050401', '20060317', '20070324', '20080313', '20080404', '20080411', '20080430', '20090324', '20100323', '20110311', '20110329', '20110413', '20120312', '20130313', '20140309', '20140330', '20160409', '20180319', '20180326', '20200326', '20210402', '20220310']


In [8]:
from datetime import datetime


if run_simulation:
    import ERA5_Tb
    default_area=[-30,50,65,89]#[5,10,65,70]
    for date in sorted(dates):
        print(date)
        yyyy = int(date[0:4])
        mm   = int(date[4:6])
        dd   = int(date[6:8])
        now = datetime.now()

        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)

        era5_processing_cls=ERA5_preprocess(yyyy,mm,dd,
                        '/home/b/b380702/pamtra/descriptorfiles/descriptor_file_ecmwf.txt',
                        outPath='/scratch/u/u300737/',area=default_area,timestep=int(hour_to_analyse)+1)
        temporary_pamtrahandler=PAMTRAhandler(era5_processing_cls)
        ## ---> which output path      
        era5_processing_cls.runCDO()
        print("CDO done")
        era5_existent,pamtra_existent=era5_processing_cls.checkfor_era5_pamtra_files()
        if not era5_existent:
            era5_processing_cls.readERA5(inPath='/scratch/u/u300737/',step=4,cut_levels=5)
            era5_processing_cls.create_pamData_dict(step=4)
            print("entire ERA5 read")
        else:
            print("Processed ERA5 already created")
            
        pamtrahandler=PAMTRAhandler(era5_processing_cls)
        if not pamtra_existent:
            print("PAMTRA TBs not there already, they should be calculated")
            if run_simulation:
                # reduce to just ocean grid points
                filter = np.empty(era5_processing_cls.pam._shape2D,dtype=bool)
                filter[:,:] = False
                filter[era5_processing_cls.pam.p['sfc_type'] == 0] = True
                era5_processing_cls.pam.filterProfiles(filter)
                pamtrahandler.runPAMTRA()
                pamtrahandler.collectERA5()
                pamtrahandler.reducePAMTRAResults(instrument='hamp')
        else:
            print("PAMTRA TBs already calculated")
#pam.r['tb'].shape
#pamtrahandler.pam.r['tb'].shape
#plotMap(pamtrahandler.pam.p['lon'],pamtrahandler.pam.p['lat'],pamtrahandler.pam.r['tb'][:,:,0,6,0,0])

19790312
Current Time = 15:54:56
Version of 2023-05-16 runned
File to check: /scratch/u/u300737/reduced_ml_19790312_12_130.nc
All ml files already calculated for  19790312
All sf files already calculated for  19790312
all IV files calculated
CDO done
Processed ERA5 already created
PAMTRA TBs already calculated
19810330
Current Time = 15:54:57
Version of 2023-05-16 runned
File to check: /scratch/u/u300737/reduced_ml_19810330_12_130.nc
All ml files already calculated for  19810330
All sf files already calculated for  19810330
all IV files calculated
CDO done
Processed ERA5 already created
PAMTRA TBs already calculated
19810424
Current Time = 15:54:58
Version of 2023-05-16 runned
File to check: /scratch/u/u300737/reduced_ml_19810424_12_130.nc
All ml files already calculated for  19810424
All sf files already calculated for  19810424
all IV files calculated
CDO done
Processed ERA5 already created
PAMTRA TBs already calculated
19820320
Current Time = 15:54:59
Version of 2023-05-16 runned
Fi

In [9]:
# move all pamtra hamp files to another directory #work
import subprocess
import glob
old_pamtra_outp_path="/scratch/u/u300737/"#pamtra_hamp_20200326_12.nc
old_file_list=glob.glob(old_pamtra_outp_path+"pamtra_hamp_*")
new_pamtra_outp_path="/work/bb1320/hdorff/pamtra_hamp_retrieval/"
if not os.path.exists(new_pamtra_outp_path):
    os.makedirs(new_pamtra_outp_path)
for file in old_file_list:
    file_name=file.split("/")[-1]
    #print(file_name)
    status = subprocess.call('cp '+file+" "+new_pamtra_outp_path+file_name, shell=True) 


pamtra_hamp_19890422_12.nc
pamtra_hamp_20140309_12.nc
pamtra_hamp_19980409_12.nc
pamtra_hamp_19830316_12.nc
pamtra_hamp_19840428_12.nc
pamtra_hamp_19930303_12.nc
pamtra_hamp_19910301_12.nc
pamtra_hamp_19970322_12.nc
pamtra_hamp_19820416_12.nc
pamtra_hamp_19840308_12.nc
pamtra_hamp_19980324_12.nc
pamtra_hamp_19810424_12.nc
pamtra_hamp_20210402_12.nc
pamtra_hamp_20080404_12.nc
pamtra_hamp_20090324_12.nc
pamtra_hamp_20120312_12.nc
pamtra_hamp_20140330_12.nc
pamtra_hamp_19840413_12.nc
pamtra_hamp_19820320_12.nc
pamtra_hamp_19790312_12.nc
pamtra_hamp_19910428_12.nc
pamtra_hamp_20110311_12.nc
pamtra_hamp_19950306_12.nc
pamtra_hamp_20180319_12.nc
pamtra_hamp_20080313_12.nc
pamtra_hamp_20130313_12.nc
pamtra_hamp_19830331_12.nc
pamtra_hamp_20070324_12.nc
pamtra_hamp_19920302_12.nc
pamtra_hamp_19950317_12.nc
pamtra_hamp_20110329_12.nc
pamtra_hamp_19930430_12.nc
pamtra_hamp_19880330_12.nc
pamtra_hamp_19940421_12.nc
pamtra_hamp_20060317_12.nc
pamtra_hamp_20200326_12.nc
pamtra_hamp_20030415_12.nc
p

In [13]:
# move all ERA5 hamp files to another directory #work
old_era5_outp_path="/scratch/u/u300737/"#pamtra_hamp_20200326_12.nc
old_file_list=glob.glob(old_pamtra_outp_path+"era5_*")
new_era5_outp_path="/work/bb1320/hdorff/pamtra_hamp_retrieval/"
if not os.path.exists(new_era5_outp_path):
    os.makedirs(new_era5_outp_path)
for file in old_file_list:
    file_name=file.split("/")[-1]
    #print(file_name)
    status = subprocess.call('cp '+file+" "+new_era5_outp_path+file_name, shell=True) 

era5_20030415_12_atmos.nc
era5_20050401_12_atmos.nc
era5_19980409_12_atmos.nc
era5_20080313_12_atmos.nc
era5_19940421_12_atmos.nc
era5_20060317_12_atmos.nc
era5_20090324_12_atmos.nc
era5_19900411_12_atmos.nc
era5_19920302_12_atmos.nc
era5_20130313_12_atmos.nc
era5_20200326_12_atmos.nc
era5_20180326_12_atmos.nc
era5_20100323_12_atmos.nc
era5_19980324_12_atmos.nc
era5_20140309_12_atmos.nc
era5_19840308_12_atmos.nc
era5_19810424_12_atmos.nc
era5_19830331_12_atmos.nc
era5_19830414_12_atmos.nc
era5_19890422_12_atmos.nc
era5_19840428_12_atmos.nc
era5_20180319_12_atmos.nc
era5_19820320_12_atmos.nc
era5_20140330_12_atmos.nc
era5_20110413_12_atmos.nc
era5_19920413_12_atmos.nc
era5_19930303_12_atmos.nc
era5_20110311_12_atmos.nc
era5_19840413_12_atmos.nc
era5_20120312_12_atmos.nc
era5_20160409_12_atmos.nc
era5_20220310_12_atmos.nc
era5_20210402_12_atmos.nc
era5_19910428_12_atmos.nc
era5_20070324_12_atmos.nc
era5_19880330_12_atmos.nc
era5_20080411_12_atmos.nc
era5_19910301_12_atmos.nc
era5_1981033

In [10]:
#era_ds=xr.open_dataset("/scratch/u/u300737/era5_"+date+"_"+hour_to_analyse+"_atmos.nc")
#era_ds.obs_height_values
pamtra_ds=xr.open_dataset(new_pamtra_outp_path+file_name)
pamtra_ds

<xarray.Dataset>
Dimensions:     (x: 477, y: 1, nout: 44, nang: 3, nfreq: 40, npol: 2)
Dimensions without coordinates: x, y, nout, nang, nfreq, npol
Data variables:
    unixtime    (x, y) float64 ...
    lat         (x, y) float64 ...
    lon         (x, y) float64 ...
    obs_height  (nout) float64 ...
    ang         (nang) float64 ...
    freq        (nfreq) float64 ...
    pol         (npol) object ...
    tb          (x, y, nout, nang, nfreq, npol) float64 ...
Attributes:
    description:  Reduced simulated brightness temperatures for hamp based on...
    models:       ERA5 + PAMTRA
    date:         19830414 12:00

In [15]:
era_ds=xr.open_dataset("/scratch/u/u300737/era5_"+date+"_"+hour_to_analyse+"_atmos.nc")
era_ds=xr.open_dataset(new_era5_outp_path+file_name)
era_ds

<xarray.Dataset>
Dimensions:     (x: 459, y: 1, nout: 45, z: 131)
Dimensions without coordinates: x, y, nout, z
Data variables: (12/16)
    unixtime    (x, y) datetime64[ns] ...
    lat         (x, y) float64 ...
    lon         (x, y) float64 ...
    obs_height  (nout) float64 ...
    sfc_slf     (x, y) float64 ...
    sfc_sif     (x, y) float64 ...
    ...          ...
    p           (x, y, z) float64 ...
    iwv         (x, y, nout) float64 ...
    lwp         (x, y, nout) float64 ...
    iwp         (x, y, nout) float64 ...
    rwp         (x, y, nout) float64 ...
    swp         (x, y, nout) float64 ...